In [1]:
import matplotlib.pyplot as plt  # plotting library
import scipy                     # scientific computnig and technical computing
import cv2                       # working with, mainly resizing, images
import numpy as np               # dealing with arrays
import glob                      # return a possibly-empty list of path names that match pathname
import os                        # dealing with directories
import pandas as pd              # providing data structures and data analysis tools
import itertools
import random
from random import shuffle       # mixing up or currently ordered data that might lead our network astray in training.
from PIL import Image
from scipy import ndimage
from skimage import io
%matplotlib inline
np.random.seed(1)

In [2]:
base_dir = os.getcwd()+"\\"

In [3]:
training_img = io.imread('images_training.tiff')
training_imgarray = np.array(training_img)

In [4]:
#validation_img = io.imread('images_test.tiff')
#validation_imgarray = np.array(validation_img)

In [5]:
df_train = pd.read_csv('descriptions_training.csv', header=None)
df_train = df_train.rename(columns={0:"imgID",1:"label",2:"c1x",3:"c1y",4:"c2x",5:"c2y"})

In [6]:
df_train_yolo = pd.DataFrame(data={0:[0], 1:[0], 2:[0], 3:[0], 4:[0], 5:[0], 6:[0], 7:[0], 8:[0], 9:[0]})
df_train_yolo = df_train_yolo.rename(columns={0:"imgID",1:"nSpots",2:"x1",3:"y1",4:"h1",5:"w1",6:"x2",7:"y2",8:"h2",9:"w2"})

In [7]:
def convert2yolo(size, center):
    dw = 1./size[0]
    dh = 1./size[1]
    x = center[0]
    y = center[1]
    # width and height will be same since spot is a circle
    w = 11  # in px
    h = 11  # in px
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [9]:
for idx, rows in df_train.iterrows():
    imageID = df_train.imgID[idx]
    nSpots = df_train.label[idx]
    #x1, y1, h1, w1, x2, y2, h2, w2 = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    if rows[1] == 0:
        x1, y1, h1, w1, x2, y2, h2, w2 = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    elif rows[1] == 1:
        x1, y1, h1, w1 = convert2yolo(training_imgarray[idx].shape, [rows[2], rows[3]])
        x2, y2, h2, w2 = np.nan, np.nan, np.nan, np.nan
    elif rows[1] == 2:
        x1, y1, h1, w1 = convert2yolo(training_imgarray[idx].shape, [rows[2], rows[3]])
        x2, y2, h2, w2 = convert2yolo(training_imgarray[idx].shape, [rows[4], rows[5]])
    temp = pd.DataFrame(data={0:[imageID], 1:[nSpots], 2:[x1], 3:[y1], 4:[h1], 5:[w1], 6:[x2], 7:[y2], 8:[h2], 9:[w2]})
    temp = temp.rename(columns={0:"imgID",1:"nSpots",2:"x1",3:"y1",4:"h1",5:"w1",6:"x2",7:"y2",8:"h2",9:"w2"})
    df_train_yolo = df_train_yolo.append(temp, ignore_index=True)
df_train_yolo = df_train_yolo[1:]

In [10]:
df_train_yolo.head(5)

,imgID,nSpots,x1,y1,h1,w1,x2,y2,h2,w2
1,0,1,0.786556,0.690551,0.458333,0.458333,NaN,NaN,NaN,NaN
2,1,1,0.423296,0.654921,0.458333,0.458333,NaN,NaN,NaN,NaN
3,2,1,0.393377,0.617913,0.458333,0.458333,NaN,NaN,NaN,NaN
4,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
label_dir = base_dir+"labels\\"

for idx, rows in df_train_yolo.iterrows():
    fname = df_train_yolo.imgID[idx]
    ftemp = open(label_dir+str(fname)+".txt","w")
    if df_train_yolo.nSpots[idx] == 0:
        ftemp.write("")
    elif df_train_yolo.nSpots[idx] == 1:
        ftemp.write("0 "+str(df_train_yolo.x1[idx])+" "+str(df_train_yolo.y1[idx])+" "
                        +str(df_train_yolo.h1[idx])+" "+str(df_train_yolo.w1[idx])+"\n")
    elif df_train_yolo.nSpots[idx] == 2:
        ftemp.write("0 "+str(df_train_yolo.x1[idx])+" "+str(df_train_yolo.y1[idx])+" "
                        +str(df_train_yolo.h1[idx])+" "+str(df_train_yolo.w1[idx])+"\n")
        ftemp.write("0 "+str(df_train_yolo.x2[idx])+" "+str(df_train_yolo.y2[idx])+" "
                        +str(df_train_yolo.h2[idx])+" "+str(df_train_yolo.w2[idx])+"\n")
    ftemp.close()